# Retrieval Augmented Generation

LLMs have a limited context window. While some LLMs have context windows as large as 2 million tokens, it's not enough to pass many large document collections (e.g. all patents, all laws, all of Wikipedia, etc.)

It's also expensive to pass all of this information every time as context.

Instead, we use a technique called "Retrieval Augmented Generation". This involves:

1. Searching for the most relevant documents using embeddings
2. Passing those along with the question to the LLM

## Tourist information

Let's create a tourist recommendation bot. The user will share their interests. The bot will recommend places to visit and explain why.

Here's the data.

> NOTE: This was generated by GPT-4o with the prompt: "Describing the top tourist attractions across popular tourist cities. Write as a table with 50 rows, one per city, with 3 columns: Country, City, Attractions. The attractions must be a brief, entity-dense paragraph mentioning the top 3 places to see and why."

In [ ]:
import pandas as pd

data = pd.DataFrame([
  ["France", "Paris", "The Eiffel Tower offers stunning city views. The Louvre houses thousands of artworks, including the Mona Lisa. Notre-Dame Cathedral is a masterpiece of Gothic architecture."],
  ["USA", "New York", "Times Square is known for its vibrant atmosphere and neon lights. Central Park provides a green escape in the city. The Statue of Liberty is a symbol of freedom and democracy."],
  ["UK", "London", "The British Museum features an extensive collection of artifacts. The Tower of London is rich in history and the Crown Jewels. Buckingham Palace is the Queen's residence and a site of royal ceremonies."],
  ["Italy", "Rome", "The Colosseum showcases ancient gladiatorial contests. Vatican City, with St. Peter's Basilica and the Sistine Chapel, is a spiritual center. The Pantheon is a well-preserved Roman temple."],
  ["Japan", "Tokyo", "Tokyo Tower offers panoramic city views. The Imperial Palace is the residence of the Emperor. Shibuya Crossing is famous for its bustling pedestrian traffic."],
  ["Australia", "Sydney", "The Sydney Opera House is an iconic architectural marvel. Sydney Harbour Bridge offers breathtaking views. Bondi Beach is perfect for sunbathing and surfing."],
  ["Spain", "Barcelona", "Sagrada Família is a unique basilica designed by Gaudí. Park Güell is a colorful, whimsical park also by Gaudí. La Rambla is a lively street full of shops and street performers."],
  ["UAE", "Dubai", "Burj Khalifa is the world's tallest building with an observation deck. The Dubai Mall offers extensive shopping and entertainment. The Palm Jumeirah is a man-made island with luxury resorts."],
  ["Turkey", "Istanbul", "Hagia Sophia is a historic architectural wonder. The Blue Mosque is renowned for its stunning blue tiles. Topkapi Palace was the residence of Ottoman sultans."],
  ["Germany", "Berlin", "Brandenburg Gate is a symbol of unity. The Berlin Wall Memorial provides insight into the Cold War era. Museum Island is home to five world-renowned museums."],
  ["Thailand", "Bangkok", "The Grand Palace is an elaborate royal complex. Wat Arun, the Temple of Dawn, offers stunning river views. Chatuchak Market is one of the largest markets in the world."],
  ["Greece", "Athens", "The Acropolis is an ancient citadel with the Parthenon. The National Archaeological Museum houses extensive Greek artifacts. Plaka is a charming historic district with narrow streets."],
  ["Brazil", "Rio de Janeiro", "Christ the Redeemer statue offers panoramic city views. Sugarloaf Mountain is another vantage point with stunning vistas. Copacabana Beach is famous for its vibrant atmosphere."],
  ["China", "Beijing", "The Great Wall is an ancient fortification offering scenic views. The Forbidden City is an imperial palace complex. Tiananmen Square is one of the largest public squares in the world."],
  ["India", "Delhi", "The Red Fort is a historic fortification. India Gate is a war memorial with lush lawns. Qutub Minar is a tall, ancient minaret with intricate carvings."],
  ["Russia", "Moscow", "The Kremlin is a fortified complex with museums and churches. Red Square features St. Basil's Cathedral with its colorful domes. The Bolshoi Theatre is renowned for its ballet and opera."],
  ["Canada", "Toronto", "The CN Tower offers panoramic city views. Royal Ontario Museum showcases art, culture, and nature. Ripley's Aquarium is home to a wide variety of marine life."],
  ["Argentina", "Buenos Aires", "The Obelisk is a historic landmark in the city center. Teatro Colón is a renowned opera house. La Boca is famous for its colorful buildings and tango culture."],
  ["Egypt", "Cairo", "The Pyramids of Giza are ancient wonders of the world. The Egyptian Museum houses an extensive collection of antiquities. Khan El Khalili is a bustling bazaar."],
  ["Mexico", "Mexico City", "The Zócalo is a historic city square. The National Museum of Anthropology offers insights into pre-Columbian heritage. Chapultepec Park is one of the largest city parks in the world."],
  ["Indonesia", "Bali", "Uluwatu Temple offers stunning sea views. Sacred Monkey Forest Sanctuary is home to hundreds of monkeys. Tanah Lot Temple is a scenic rock formation and pilgrimage site."],
  ["South Africa", "Cape Town", "Table Mountain provides panoramic city views. Robben Island is a historic prison where Nelson Mandela was held. Victoria & Alfred Waterfront is a vibrant shopping and entertainment area."],
  ["Singapore", "Singapore", "Marina Bay Sands is a luxury resort with an iconic rooftop pool. Gardens by the Bay features futuristic supertrees. Sentosa Island offers beaches and theme parks."],
  ["Netherlands", "Amsterdam", "The Anne Frank House is a poignant historical site. The Van Gogh Museum houses an extensive collection of the artist's works. The Rijksmuseum showcases Dutch art and history."],
  ["South Korea", "Seoul", "Gyeongbokgung Palace is a grand royal palace. N Seoul Tower offers panoramic city views. Myeongdong is a bustling shopping district."],
  ["Malaysia", "Kuala Lumpur", "Petronas Towers are iconic twin skyscrapers. Batu Caves is a limestone hill with a series of caves and temples. KL Tower offers panoramic views of the city."],
  ["Vietnam", "Hanoi", "Ho Chi Minh Mausoleum is a historic site. The Old Quarter is known for its bustling streets and markets. The Temple of Literature is a well-preserved example of traditional Vietnamese architecture."],
  ["Portugal", "Lisbon", "Belém Tower is a historic fortress. Jerónimos Monastery is a stunning example of Manueline architecture. Alfama is a charming district with narrow streets and fado music."],
  ["New Zealand", "Auckland", "Sky Tower offers panoramic city views. Waiheke Island is known for its vineyards and beaches. Auckland War Memorial Museum provides insights into New Zealand's history and culture."],
  ["Austria", "Vienna", "Schönbrunn Palace is a grand baroque palace. St. Stephen's Cathedral is a Gothic masterpiece. The Hofburg is a historic palace complex."],
  ["Czech Republic", "Prague", "Charles Bridge is a historic bridge with stunning views. Prague Castle is a sprawling castle complex. Old Town Square features the Astronomical Clock."],
  ["Hungary", "Budapest", "Buda Castle is a historic castle and palace complex. The Hungarian Parliament Building is a stunning example of neo-Gothic architecture. The Széchenyi Thermal Bath is a popular spa destination."],
  ["Ireland", "Dublin", "Trinity College is home to the Book of Kells. Dublin Castle is a historic castle complex. Temple Bar is a vibrant cultural quarter."],
  ["Sweden", "Stockholm", "The Vasa Museum houses a 17th-century ship. Gamla Stan is the charming old town with cobblestone streets. The Royal Palace is one of the largest palaces in Europe."],
  ["Norway", "Oslo", "The Vigeland Park features a large collection of sculptures. The Viking Ship Museum showcases well-preserved Viking ships. The Royal Palace is the official residence of the Norwegian monarch."],
  ["Denmark", "Copenhagen", "Tivoli Gardens is a historic amusement park. Nyhavn is a picturesque waterfront district. The Little Mermaid statue is a symbol of the city."],
  ["Belgium", "Brussels", "The Grand Place is a stunning central square. Atomium is a unique building and museum. Manneken Pis is a famous statue and cultural symbol."],
  ["Finland", "Helsinki", "Suomenlinna is a historic sea fortress. The Helsinki Cathedral is a neoclassical landmark. Market Square is a bustling market by the harbor."],
  ["Poland", "Warsaw", "The Royal Castle is a historic palace. The Warsaw Uprising Museum offers insights into World War II history. Łazienki Park is a large and picturesque park."],
  ["Chile", "Santiago", "San Cristóbal Hill offers panoramic city views. Plaza de Armas is the historic center of the city. La Moneda Palace is the seat of the President of Chile."],
  ["Colombia", "Bogotá", "Monserrate offers panoramic city views. The Gold Museum houses a vast collection of pre-Columbian gold artifacts. La Candelaria is the historic district with colorful colonial buildings."],
  ["Peru", "Lima", "The Larco Museum showcases pre-Columbian art. Plaza Mayor is the historic center of the city. Miraflores is a vibrant district with shopping and coastal views."],
  ["Switzerland", "Zurich", "Lake Zurich offers scenic boat rides. The Old Town features medieval buildings and narrow streets. The Swiss National Museum provides insights into Swiss culture and history."],
  ["Austria", "Salzburg", "Hohensalzburg Fortress is a well-preserved castle. Mirabell Palace and Gardens are a beautiful baroque ensemble. Mozart's Birthplace is now a museum dedicated to the composer."],
  ["Portugal", "Porto", "Ribeira is a charming riverside district. The Livraria Lello is one of the most beautiful bookstores in the world. The Porto Cathedral is a Romanesque landmark."],
  ["Scotland", "Edinburgh", "Edinburgh Castle offers stunning views and history. The Royal Mile is a historic street with shops and attractions. Arthur's Seat is a hill providing panoramic city views."],
  ["Iceland", "Reykjavik", "The Hallgrimskirkja Church is a striking modern cathedral. The Blue Lagoon is a geothermal spa. Harpa is a stunning concert hall and conference center."],
  ["Croatia", "Dubrovnik", "The Old Town is a UNESCO World Heritage site with medieval walls. The Dubrovnik Cable Car offers panoramic views. Lovrijenac Fort is a historic fortress overlooking the sea."],
  ["Monaco", "Monaco", "The Monte Carlo Casino is an iconic gambling venue. The Prince's Palace is the official residence of the ruling prince. The Oceanographic Museum is renowned for its marine exhibits."],
], columns=["country", "city", "reason"])

data

,country,city,reason
0,France,Paris,The Eiffel Tower offers stunning city views. T...
1,USA,New York,Times Square is known for its vibrant atmosphe...
2,UK,London,The British Museum features an extensive colle...
3,Italy,Rome,The Colosseum showcases ancient gladiatorial c...
4,Japan,Tokyo,Tokyo Tower offers panoramic city views. The I...
5,Australia,Sydney,The Sydney Opera House is an iconic architectu...
6,Spain,Barcelona,Sagrada Família is a unique basilica designed ...
7,UAE,Dubai,Burj Khalifa is the world's tallest building w...
8,Turkey,Istanbul,Hagia Sophia is a historic architectural wonde...
9,Germany,Berlin,Brandenburg Gate is a symbol of unity. The Ber...


## Pick most similar docs using embeddings

When the user shares an interest, e.g. "history" or "night life" or "relaxation", we find the documents (i.e. rows in the data) that are most similar to their interest.

To do this, let's generate the embeddings for these.

In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('Alibaba-NLP/gte-large-en-v1.5', trust_remote_code=True)
embeddings = model.encode(data['reason'].tolist())
print(embeddings)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when p

[[-0.5180088  -0.08570045 -0.39213437 ...  0.6789873  -0.4922544
  -1.2848637 ]
 [ 0.04371696 -0.39617836 -0.7663586  ...  1.32225    -1.1490729
  -1.0119119 ]
 [-0.37237552 -1.1430222   0.50798357 ...  0.3442477  -0.2090202
  -1.2785453 ]
 ...
 [ 0.15203829 -0.23567551  0.50869864 ...  0.21508119 -0.6404027
  -0.89880157]
 [-0.2935335  -0.3791502   0.96748674 ...  0.08428381  0.09384883
  -1.0101628 ]
 [ 0.06181398 -0.3188959   0.1018238  ...  0.77323705 -0.85136956
  -1.0803283 ]]


Now we can get the embedding for their interest, e.g. "history" or "night life".

In [ ]:
# Query the collection for most similar documents to "history"
query_embedding = model.encode(["history"])

query_embedding

array([[-0.53353566, -0.59643537, -0.8621425 , ...,  0.15550502,
        -0.50911534, -0.06096759]], dtype=float32)

To find the most similar, we can take cosine similarity.

In [ ]:
# Find the most similar embedding to "history"
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(embeddings, query_embedding).flatten()
similarity

array([0.6036385 , 0.53626347, 0.57137084, 0.609784  , 0.51776916,
       0.4628374 , 0.48380017, 0.41624236, 0.558047  , 0.5867095 ,
       0.53348005, 0.5777191 , 0.47099942, 0.58915097, 0.5981548 ,
       0.58122635, 0.49066585, 0.52144444, 0.5747149 , 0.57797486,
       0.4431392 , 0.5286194 , 0.3883853 , 0.5211549 , 0.4454609 ,
       0.5186015 , 0.53541476, 0.50616926, 0.5009943 , 0.5684246 ,
       0.5765037 , 0.5256419 , 0.6028552 , 0.57750523, 0.56394243,
       0.5180951 , 0.5689187 , 0.5532517 , 0.54774874, 0.5175859 ,
       0.5022483 , 0.52558804, 0.5197584 , 0.57155985, 0.49122572,
       0.49656177, 0.5159164 , 0.54573995, 0.54089165], dtype=float32)

Then we pick the top 3 rows among these.

In [ ]:
# Get the most similar rows
import numpy as np

most_similar = np.argsort(similarity)[-3:][::-1]

data.iloc[most_similar]

,country,city,reason
3,Italy,Rome,The Colosseum showcases ancient gladiatorial c...
0,France,Paris,The Eiffel Tower offers stunning city views. T...
32,Ireland,Dublin,Trinity College is home to the Book of Kells. ...


We can abstract this into a function and pass it any user interest.

In [ ]:
# Let's get the most similar rows to "relaxation"

def most_similar(query):
    query_embedding = model.encode([query])
    similarity = cosine_similarity(embeddings, query_embedding).flatten()
    most_similar = np.argsort(similarity)[-3:][::-1]
    return data.iloc[most_similar]

most_similar("relaxation")

,country,city,reason
46,Iceland,Reykjavik,The Hallgrimskirkja Church is a striking moder...
22,Singapore,Singapore,Marina Bay Sands is a luxury resort with an ic...
21,South Africa,Cape Town,Table Mountain provides panoramic city views. ...


In [ ]:
# What if we wanted night life?

most_similar("night life")

,country,city,reason
1,USA,New York,Times Square is known for its vibrant atmosphe...
21,South Africa,Cape Town,Table Mountain provides panoramic city views. ...
17,Argentina,Buenos Aires,The Obelisk is a historic landmark in the city...


## Ask an LLM to answer the question from context

Now that we have the *most relevant* documents to pass to the LLM, we can ask the LLM to answer the question only using this context.

In [ ]:
# Now let's just pass these 3 to an LLM and ask it to answer the question

import json
import requests
from google.colab import userdata

api_key = userdata.get('OPENAI_API_KEY')

def pick_tourist_city_for(interest):
    cities = most_similar(interest)
    response = requests.post(
        "https://api.openai.com/v1/chat/completions",
        headers={"Authorization": f"Bearer {api_key}"},
        json={
            "model": "gpt-3.5-turbo",
            "messages": [
                {"role": "system", "content": f"I am a tourist interested in {interest}. Based ONLY on the information below, BRIEFLY suggest where I should go and why."},
                {"role": "user", "content": cities.to_json(orient="records")},
            ]
        }
    )
    result = response.json()
    return result["choices"][0]["message"]["content"]


pick_tourist_city_for("history")

'You should visit Rome, Italy. The city is rich in history with attractions like the Colosseum, Vatican City, and the Pantheon, all showcasing ancient Roman architecture and culture.'

In [ ]:
pick_tourist_city_for("night life")

'You should visit Buenos Aires, Argentina. The city is famous for its vibrant night life in areas like La Boca, known for its colorful buildings and tango culture, as well as the historic landmark of Teatro Colón, where you can experience renowned opera performances.'

In [ ]:
pick_tourist_city_for("relaxation")

'You should visit the Blue Lagoon geothermal spa in Reykjavik, Iceland. It offers a unique and relaxing experience in the mineral-rich waters, surrounded by stunning natural landscapes.'

## Next steps

This is a simple implementation of the retrieval-augmented generation approach.

When implementing it for a large number of documents in real life, you are likely to need:

1. A [Vector Database](https://github.com/dangkhoasdc/awesome-vector-database) that can store the documents and the embeddings on disk rather than memory, and efficiently find just the most similar documents.
2. Maybe a re-ranker that can sort the most relevant documents in line with the question, so that the LLM gets a better input
3. A better prompt that ensures that the answer is relevant and only from the context (not from the LLM's past training.)